# Accesibilidad a áreas verdes en la Zona Metropolitana del Valle de México

En este taller vamos a calcular diferentes modelos de accesibilidad a áreas verdes. La idea es ir trabajando de lo más simple a los más complejo. Empezaremos con un modelo de conectividad basado sólo en distancias entre centroides e iremos complicando las cosas a partir de ahí, primero usando conectividad de red...

## Preparación de datos

Como siempre, lo primero que tenemos que hacer es preparar los datos. En este caso vamos a usar el marco cartográfico del Censo de 2020 a nivel AGEB y una cartografía de áreas verdes que preparé para un proyecto integrando datos de INEGI y de OSM.

La cartografía de áreas verdes viene en un geopackage para todo el país y la cartografía censal viene en shapes para la CDMX y el Estado de México. Para cortar todo con la silueta de la ZMVM tenemos un shaper con el límite metropolitano.

Los datos para obtener la demanda (población, en principio) vienen en csvs que tenemos que unir con la geometría de las manzanas. Eso lo haremos después de tener los recortes para economizar un poco.

In [1]:
import pandas as pd
import geopandas as gpd

### Leer datos

In [2]:
cdmx = gpd.read_file("datos/agebs_cdmx_2020.zip")
edomex = gpd.read_file("datos/agebs_edomex_2020.zip")
areas_verdes = gpd.read_file("datos/parques.gpkg", layer='parques')
limite_metropolitano = gpd.read_file("datos/limite_metropolitano.zip")

Hacemos un merge de las agebs para tenerlas en un solo dataframe

In [3]:
agebs_todas = pd.concat([cdmx, edomex])
agebs_todas

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry
0,0901000011716,09,010,0001,1716,"POLYGON ((2787237.541 816989.461, 2787288.728 ..."
1,0901000012150,09,010,0001,2150,"POLYGON ((2794154.458 823013.444, 2794155.774 ..."
2,0901000011133,09,010,0001,1133,"POLYGON ((2795690.723 820050.788, 2795684.238 ..."
3,0901000011307,09,010,0001,1307,"POLYGON ((2792584.475 815678.668, 2792624.325 ..."
4,0901000010281,09,010,0001,0281,"POLYGON ((2788845.392 823526.074, 2788840.549 ..."
...,...,...,...,...,...,...
4315,151250001004A,15,125,0001,004A,"POLYGON ((2808390.934 857912.251, 2808614.795 ..."
4316,1512500010016,15,125,0001,0016,"POLYGON ((2808318.468 857265.867, 2808312.961 ..."
4317,1512500010020,15,125,0001,0020,"POLYGON ((2807637.181 858200.578, 2807639.598 ..."
4318,1512500070054,15,125,0007,0054,"POLYGON ((2806574.677 856458.207, 2806484.183 ..."


### Proyectar
Para poder cortar las geometrías y trabajarlas todas juntas necesitamos que estén en la misma proyección

In [12]:
print(cdmx.crs)
print(edomex.crs)
print(areas_verdes.crs)
print(limite_metropolitano.crs)

PROJCS["MEXICO_ITRF_2008_LCC",GEOGCS["ITRF2008",DATUM["International_Terrestrial_Reference_Frame_2008",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1061"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AUTHORITY["EPSG","8999"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
PROJCS["MEXICO_ITRF_2008_LCC",GEOGCS["ITRF2008",DATUM["International_Terrestrial_Reference_Frame_2008",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1061"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AUTHORITY["EPSG","8999"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridia

Las primeras dos están en la proyección nueva de INEGI, las áreas verdes en la anterior y el límite metropolitano está en 32614. Tenemos que poner todas en la misma proyección, podría ser cualquiera de las planas (vamos a medir longitudes). Usemos la nueva del INEGI

In [15]:
areas_verdes = areas_verdes.to_crs(cdmx.crs)
limite_metropolitano = limite_metropolitano.to_crs(cdmx.crs)

### Cortar con el límite metropolitano

In [21]:
from geopandas.tools import sjoin
areas_verdes = sjoin(areas_verdes, limite_metropolitano)
agebs_zmvm = sjoin(agebs_todas, limite_metropolitano)

### Áreas y centroides

Como vamos a trabajar la accesibilidad en primera instancia sólo entre los centroides de las agebs y las de las áreas verdes, entonces vamos a necesitar dataframes con el área de los parques (que vamos a usar como la oferta en el modelo de accesibilidad) y con los centroides de las agebs y su población total (que va a ser la demanda).

**Nota:** vamos a hacer una copia de las capas para usdar los centroides, en caso de que queramos después usar las geometrías originales

In [31]:
areas_verdes['area'] = areas_verdes.geometry.area
areas_verdes_centroides = areas_verdes.copy()
areas_verdes_centroides['geometry'] = areas_verdes_centroides.centroid
# Nos quedamos sólo con los campos que queremos
areas_verdes_centroides = areas_verdes_centroides.loc[:, ['geometry', 'area']]
agebs_zmvm_centroides = agebs_zmvm.copy()
agebs_zmvm_centroides['geometry'] = agebs_zmvm_centroides.centroid
# Nos quedamos sólo con los campos que queremos
agebs_zmvm_centroides = agebs_zmvm_centroides.loc[:, ['CVEGEO', 'geometry']]

### Datos de población
Para modelar la demanda necesitamos ahora unir a las agebs los datos de población, esos vienen en csvs por estado. Lo primero es leerlos y juntarlos en un solo dataframe

In [35]:
datos_cdmx = pd.read_csv("datos/conjunto_de_datos_ageb_urbana_09_cpv2020.zip", dtype={'ENTIDAD':str, 'MUN':str, 'LOC':str, 'AGEB':str})
datos_edomex = pd.read_csv("datos/conjunto_de_datos_ageb_urbana_15_cpv2020.zip", dtype={'ENTIDAD':str, 'MUN':str, 'LOC':str, 'AGEB':str})
datos_zmvm = pd.concat([datos_cdmx, datos_edomex])
# Construimos una columna con la clave de ageb
datos_zmvm['CVEGEO'] = datos_zmvm['ENTIDAD'] + datos_zmvm['MUN'] + datos_zmvm['LOC'] + datos_zmvm['AGEB']
# Nos quedamos sólo con las columnas que vamos a usar (sólo queremos los datos de población)
datos_zmvm = datos_zmvm.loc[:, ['CVEGEO', 'POBTOT']]

In [36]:
datos_zmvm

,CVEGEO,POBTOT
0,0900000000000,9209944
1,0900200000000,432205
2,0900200010000,432205
3,0900200010010,3183
4,0900200010010,159
...,...,...
141525,1512500070069,11
141526,1512500070069,19
141527,1512500070069,3
141528,1512500070069,12
